<a href="https://colab.research.google.com/github/mavy731/UIUC-MSFE-FALL20-FIN553/blob/main/Assignment1_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

Spot = 36   # stock price
σ = 0.2     # stock volatility
K = 40      # strike price
r = 0.06    # risk free rate
n = 20000  # Number of simualted paths
m = 50      # number of exercise dates
T = 1       # maturity
order = 6   # Polynmial order
Δt = T / m  # interval between two exercise dates


def create_polynomials():
    model = tf.keras.Sequential([tf.keras.layers.Dense(1)])
    return model


networks = {t: create_polynomials() for t in range(1, m)}


# simulates the stock price evolution
def advance(S, r, σ, Δt, n):
    dB = tf.sqrt(Δt) * tf.random.normal([n])
    out = S + r * S * Δt + σ * S * dB
    return out

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# LSMC algorithm

tf.random.set_seed(0)
S0 = Spot * tf.ones([n],dtype=tf.float32)
S = {0: S0}

for t in range(m):
  S[t + 1] = advance(S[t], r, σ, Δt, n)

discount = tf.exp(-r * Δt)
CFL = {t: tf.maximum(0., K - S[t]) for t in range(m + 1)}
value_tp1 = CFL[m] * discount
CV = {m: tf.zeros_like(S[0])}

for t in range(m - 1, 0, -1):
  f=networks[t]
  f.compile(loss='mse',optimizer=optimizer)
  f.fit(S[t],value_tp1,epochs=1)
  CV[t]=f(S[t])
  value_t = tf.where(CFL[t] > CV[t], CFL[t], value_tp1)
  value_tp1 = discount * value_t

POF = {t: tf.where(CV[t] < CFL[t], CFL[t], 0) for t in range(1, m + 1)}
POF=tf.stack(list(POF.values()), axis=0)
idx_payoffs = tf.argmax(POF > 0, axis=0)
FPOF = tf.transpose(tf.one_hot(idx_payoffs, m)) * POF
m_range = tf.reshape(tf.constant(range(0, m),dtype=tf.float32),[-1,1])
dFPOF = FPOF * tf.exp(-r * m_range * Δt)
PRICE = tf.math.reduce_sum(dFPOF)/ n



In [ ]:
print(PRICE)